In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/uber-pickups-in-new-york-city/other-Carmel_B00256.csv
/kaggle/input/uber-pickups-in-new-york-city/other-FHV-services_jan-aug-2015.csv
/kaggle/input/uber-pickups-in-new-york-city/uber-raw-data-may14.csv
/kaggle/input/uber-pickups-in-new-york-city/uber-raw-data-janjune-15.csv
/kaggle/input/uber-pickups-in-new-york-city/other-Firstclass_B01536.csv
/kaggle/input/uber-pickups-in-new-york-city/other-Dial7_B00887.csv
/kaggle/input/uber-pickups-in-new-york-city/other-Federal_02216.csv
/kaggle/input/uber-pickups-in-new-york-city/other-American_B01362.csv
/kaggle/input/uber-pickups-in-new-york-city/uber-raw-data-jul14.csv
/kaggle/input/uber-pickups-in-new-york-city/uber-raw-data-apr14.csv
/kaggle/input/uber-pickups-in-new-york-city/other-Diplo_B01196.csv
/kaggle/input/uber-pickups-in-new-york-city/uber-raw-data-sep14.csv
/kaggle/input/uber-pickups-in-new-york-city/uber-raw-data-jun14.csv
/kaggle/input/uber-pickups-in-new-york-city/Uber-Jan-Feb-FOIL.csv
/kaggle/input/uber-pickups-in

In [3]:
#!pip install pyspark
import pyspark.sql.functions as sql_func
from pyspark.sql.types import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

sc = SparkContext('local')
ss = SparkSession(sc)

     |████████████████████████████████| 215.7MB 42kB/s s eta 0:00:01
     |████████████████████████████████| 204kB 40.8MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216131250 sha256=a963af8463cebdfcea55558e16c7eb80713943435be3c5a88ed2aa4e80daceec
  Stored in directory: /tmp/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [4]:
uber_apr14 = pd.read_csv("/kaggle/input/uber-pickups-in-new-york-city/uber-raw-data-apr14.csv")
uber_may14 = pd.read_csv("/kaggle/input/uber-pickups-in-new-york-city/uber-raw-data-may14.csv")
uber_jun14 = pd.read_csv("/kaggle/input/uber-pickups-in-new-york-city/uber-raw-data-jun14.csv")
uber_jul14 = pd.read_csv("/kaggle/input/uber-pickups-in-new-york-city/uber-raw-data-jul14.csv")
uber_aug14 = pd.read_csv("/kaggle/input/uber-pickups-in-new-york-city/uber-raw-data-aug14.csv")
uber_sep14 = pd.read_csv("/kaggle/input/uber-pickups-in-new-york-city/uber-raw-data-sep14.csv")

In [5]:
for df in [uber_apr14,uber_may14,uber_jun14,uber_jul14,uber_aug14,uber_sep14]:
    df['Date/Time']= pd.to_datetime(df['Date/Time'])


In [13]:
uber_apr14 = uber_apr14.rename(columns={"Date/Time": "Datetime"})
uber_may14 = uber_may14.rename(columns={"Date/Time": "Datetime"})
uber_jun14 = uber_jun14.rename(columns={"Date/Time": "Datetime"})
uber_jul14 = uber_jul14.rename(columns={"Date/Time": "Datetime"})
uber_aug14 = uber_aug14.rename(columns={"Date/Time": "Datetime"})
uber_sep14 = uber_sep14.rename(columns={"Date/Time": "Datetime"})

**1. Analyse des clusters mensuellement **

In [14]:
data_schema = StructType([
    StructField('Datetime',TimestampType(), False),
    StructField('Lat',DoubleType(), False),
    StructField('Lon',DoubleType(), False),
    StructField('Base',StringType(), False)
    ])

In [ ]:
df_apr14 = ss.createDataFrame(uber_apr14, schema = data_schema)
df_apr14 = df_apr14.rdd.map(lambda x:(Vectors.dense(x[1:3]), x[0:1])).toDF(["features", "label"])

df_may14 = ss.createDataFrame(uber_may14, schema = data_schema)
df_may14 = df_may14.rdd.map(lambda x:(Vectors.dense(x[1:3]), x[0:1])).toDF(["features", "label"])

df_jun14 = ss.createDataFrame(uber_jun14, schema = data_schema)
df_jun14 = df_jun14.rdd.map(lambda x:(Vectors.dense(x[1:3]), x[0:1])).toDF(["features", "label"])

df_jul14 = ss.createDataFrame(uber_jul14, schema = data_schema)
df_jul14 = df_jul14.rdd.map(lambda x:(Vectors.dense(x[1:3]), x[0:1])).toDF(["features", "label"])

df_aug14 = ss.createDataFrame(uber_aug14, schema = data_schema)
df_aug14 = df_aug14.rdd.map(lambda x:(Vectors.dense(x[1:3]), x[0:1])).toDF(["features", "label"])

df_sep14 = ss.createDataFrame(uber_sep14, schema = data_schema)
df_sep14 = df_sep14.rdd.map(lambda x:(Vectors.dense(x[1:3]), x[0:1])).toDF(["features", "label"])

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator 
for k in [2,3,4,5,6,7,8,9,10]:
    kmeans = KMeans().setK(k).setSeed(1)
    model = kmeans.fit(df_apr14)

    predictions = model.transform(df_apr14)

    evaluator = ClusteringEvaluator()

    silhouette = evaluator.evaluate(predictions)
    print("Le coeficient de silhouette pour "+str(k)+" clusters est :" + str(silhouette))

    # Shows the result.
    centers = model.clusterCenters()
    print("Les centres de Clusters pour k = "+ str(k) + " du Mois Avril 2014 :")
    for center in centers:
        print(center)

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator 
for k in [2,3,4]:
    kmeans = KMeans().setK(k).setSeed(1)
    model = kmeans.fit(df_may14)

    predictions = model.transform(df_may14)

    evaluator = ClusteringEvaluator()

    silhouette = evaluator.evaluate(predictions)
    print("Le coeficient de silhouette pour "+str(k)+" clusters est :" + str(silhouette))

    # Shows the result.
    centers = model.clusterCenters()
    print("Les centres de Clusters pour k = "+ str(k) + " du Mois Mai 2014 :")
    for center in centers:
        print(center)
    print('*'*50)
    
print('*'*150)

for k in [2,3,4]:
    kmeans = KMeans().setK(k).setSeed(1)
    model = kmeans.fit(df_jun14)

    predictions = model.transform(df_jun14)

    evaluator = ClusteringEvaluator()

    silhouette = evaluator.evaluate(predictions)
    print("Le coeficient de silhouette pour "+str(k)+" clusters est :" + str(silhouette))

    # Shows the result.
    centers = model.clusterCenters()
    print("Les centres de Clusters pour k = "+ str(k) + " du Mois Juin 2014 :")
    for center in centers:
        print(center)
    print('*'*50)

print('*'*150)

for k in [2,3,4]:
    kmeans = KMeans().setK(k).setSeed(1)
    model = kmeans.fit(df_jul14)

    predictions = model.transform(df_jul14)

    evaluator = ClusteringEvaluator()

    silhouette = evaluator.evaluate(predictions)
    print("Le coeficient de silhouette pour "+str(k)+" clusters est :" + str(silhouette))

    # Shows the result.
    centers = model.clusterCenters()
    print("Les centres de Clusters pour k = "+ str(k) + " du Mois Juillet 2014 :")
    for center in centers:
        print(center)
    print('*'*50)

print('*'*150)   

for k in [2,3,4]:
    kmeans = KMeans().setK(k).setSeed(1)
    model = kmeans.fit(df_aug14)

    predictions = model.transform(df_aug14)

    evaluator = ClusteringEvaluator()

    silhouette = evaluator.evaluate(predictions)
    print("Le coeficient de silhouette pour "+str(k)+" clusters est :" + str(silhouette))

    # Shows the result.
    centers = model.clusterCenters()
    print("Les centres de Clusters pour k = "+ str(k) + " du Mois Août 2014 :")
    for center in centers:
        print(center)
    print('*'*50)
print('*'*150)

for k in [2,3,4]:
    kmeans = KMeans().setK(k).setSeed(1)
    model = kmeans.fit(df_sep14)

    predictions = model.transform(df_sep14)

    evaluator = ClusteringEvaluator()

    silhouette = evaluator.evaluate(predictions)
    print("Le coeficient de silhouette pour "+str(k)+" clusters est :" + str(silhouette))

    # Shows the result.
    centers = model.clusterCenters()
    print("Les centres de Clusters pour k = "+ str(k) + " du Mois Septembre 2014 :")
    for center in centers:
        print(center)
    print('*'*50)
print('*'*150)

In [37]:
#!pip install folium
import folium

m= folium.Map(location=[40.7808, -73.9772])

#AVRIL
m.add_child(folium.Marker(location=[40.72439244,-73.81898693],tooltip = 'April_2014',line_color='#f20202',fill_color='#f20202', radius=20))
m.add_child(folium.Marker(location=[40.74090072,-73.98586964],tooltip = 'April_2014',line_color='#f20202',fill_color='#f20202', radius=20))

#MAI
m.add_child(folium.Marker(location=[40.7405209,-73.97270301],tooltip = 'May_2014',fill=True,
                        icon=folium.Icon(color='green'), radius=20))
m.add_child(folium.Marker(location=[40.69615375,-74.20017924],tooltip = 'May_2014',fill=True,
                        icon=folium.Icon(color='green'), radius=20))

#JUIN
m.add_child(folium.Marker(location=[40.74158986,-73.9827794],tooltip = 'June_2014', icon=folium.Icon(color='purple'), radius=20))
m.add_child(folium.Marker(location=[40.72436591,-73.81393593],tooltip = 'June_2014', icon=folium.Icon(color='purple'), radius=20))
m.add_child(folium.Marker(location=[40.65242375,-74.42285033],tooltip = 'June_2014', icon=folium.Icon(color='purple'), radius=20))
m.add_child(folium.Marker(location=[40.69657791,-74.17792873],tooltip = 'June_2014', icon=folium.Icon(color='purple'), radius=20))

#JUILLET
m.add_child(folium.Marker(location=[40.73973265,-73.97003691],tooltip = 'July_2014',icon=folium.Icon(color='darkblue'), radius=20))
m.add_child(folium.Marker(location=[40.68216694,-74.19535444],tooltip = 'July_2014',icon=folium.Icon(color='darkblue'), radius=20))

#AOUT
m.add_child(folium.Marker(location=[40.73845933,-73.98355468],tooltip = 'Aug_2014',icon=folium.Icon(color='red'), radius=20))
m.add_child(folium.Marker(location=[40.72973808,-73.81141036],tooltip = 'Aug_2014',icon=folium.Icon(color='red'), radius=20))

#SEPTEMBRE
m.add_child(folium.Marker(location=[40.73986393,-73.9839981],tooltip = 'Sep_2014',icon=folium.Icon(color='cadetblue'),  radius=20))
m.add_child(folium.Marker(location=[40.73096738,-73.81534577],tooltip = 'Sep_2014',icon=folium.Icon(color='cadetblue'), radius=20))

m

In [20]:
from pyspark.sql import SQLContext, Row
sqlCon = SQLContext(sc)
apr14 = ss.createDataFrame(uber_apr14, schema = data_schema)
apr14.registerTempTable("df_apr14")

mai14 = ss.createDataFrame(uber_may14, schema = data_schema)
mai14.registerTempTable("df_may14")

jun14 = ss.createDataFrame(uber_jun14, schema = data_schema)
jun14.registerTempTable("df_jun14")

jul14 = ss.createDataFrame(uber_jul14, schema = data_schema)
jul14.registerTempTable("df_jul14")

aug14 = ss.createDataFrame(uber_aug14, schema = data_schema)
aug14.registerTempTable("df_aug14")

sep14 = ss.createDataFrame(uber_sep14, schema = data_schema)
sep14.registerTempTable("df_sep14")

sqlCon.sql("SELECT Lat,Lon,COUNT(*) FROM df_apr14 GROUP BY Lat, Lon ORDER BY Count(Datetime) DESC").show(10)
sqlCon.sql("SELECT Lat,Lon,COUNT(*) FROM df_may14 GROUP BY Lat, Lon ORDER BY Count(Datetime) DESC").show(10)
sqlCon.sql("SELECT Lat,Lon,COUNT(*) FROM df_jun14 GROUP BY Lat, Lon ORDER BY Count(Datetime) DESC").show(10)
sqlCon.sql("SELECT Lat,Lon,COUNT(*) FROM df_jul14 GROUP BY Lat, Lon ORDER BY Count(Datetime) DESC").show(10)
sqlCon.sql("SELECT Lat,Lon,COUNT(*) FROM df_aug14 GROUP BY Lat, Lon ORDER BY Count(Datetime) DESC").show(10)
sqlCon.sql("SELECT Lat,Lon,COUNT(*) FROM df_sep14 GROUP BY Lat, Lon ORDER BY Count(Datetime) DESC").show(10)

sqlCon.sql("SELECT hour(df_apr14.Datetime),count(Datetime) FROM df_apr14 GROUP BY hour(df_apr14.Datetime) ORDER BY hour(df_apr14.Datetime) ").show(24)
sqlCon.sql("SELECT hour(df_may14.Datetime),count(Datetime) FROM df_may14 GROUP BY hour(df_may14.Datetime) ORDER BY hour(df_may14.Datetime) ").show(24)
sqlCon.sql("SELECT hour(df_jun14.Datetime),count(Datetime) FROM df_jun14 GROUP BY hour(df_jun14.Datetime) ORDER BY hour(df_jun14.Datetime) ").show(24)
sqlCon.sql("SELECT hour(df_jul14.Datetime),count(Datetime) FROM df_jul14 GROUP BY hour(df_jul14.Datetime) ORDER BY hour(df_jul14.Datetime) ").show(24)
sqlCon.sql("SELECT hour(df_aug14.Datetime),count(Datetime) FROM df_aug14 GROUP BY hour(df_aug14.Datetime) ORDER BY hour(df_aug14.Datetime) ").show(24)
sqlCon.sql("SELECT hour(df_sep14.Datetime),count(Datetime) FROM df_sep14 GROUP BY hour(df_sep14.Datetime) ORDER BY hour(df_sep14.Datetime) ").show(24)

sqlCon.sql("SELECT Lat, Lon, count(Datetime) FROM df_apr14 WHERE hour(df_apr14.Datetime)<10 AND hour(df_apr14.Datetime)>7 GROUP BY Lat, Lon, hour(df_apr14.Datetime) ORDER BY count(Datetime) DESC ").show(6)
sqlCon.sql("SELECT Lat, Lon, count(Datetime) FROM df_may14 WHERE hour(df_may14.Datetime)<10 AND hour(df_may14.Datetime)>7 GROUP BY Lat, Lon, hour(df_may14.Datetime) ORDER BY count(Datetime) DESC ").show(6)
sqlCon.sql("SELECT Lat, Lon, count(Datetime) FROM df_jun14 WHERE hour(df_jun14.Datetime)<10 AND hour(df_jun14.Datetime)>7 GROUP BY Lat, Lon, hour(df_jun14.Datetime) ORDER BY count(Datetime) DESC ").show(6)
sqlCon.sql("SELECT Lat, Lon, count(Datetime) FROM df_jul14 WHERE hour(df_jul14.Datetime)<10 AND hour(df_jul14.Datetime)>7 GROUP BY Lat, Lon, hour(df_jul14.Datetime) ORDER BY count(Datetime) DESC ").show(6)
sqlCon.sql("SELECT Lat, Lon, count(Datetime) FROM df_aug14 WHERE hour(df_aug14.Datetime)<10 AND hour(df_aug14.Datetime)>7 GROUP BY Lat, Lon, hour(df_aug14.Datetime) ORDER BY count(Datetime) DESC ").show(6)
sqlCon.sql("SELECT Lat, Lon, count(Datetime) FROM df_sep14 WHERE hour(df_sep14.Datetime)<10 AND hour(df_sep14.Datetime)>7 GROUP BY Lat, Lon, hour(df_sep14.Datetime) ORDER BY count(Datetime) DESC ").show(6)

In [52]:
latitude = [40.72439244,40.74090072,40.7405209,40.69615375,40.74158986,40.72436591,40.65242375,40.69657791,40.73973265,40.68216694,40.73845933,40.72973808,40.73986393,40.73096738]
longitude = [-73.81898693,-73.98586964,-73.97270301,-74.20017924,-73.9827794,-73.81393593,-74.42285033,-74.17792873,-73.97003691,-74.19535444,-73.98355468,-73.81141036,-73.9839981,-73.81534577]


,Datetime,Lat,Lon,Base
0,2014-08-01 00:03:00,40.7366,-73.9906,B02512
1,2014-08-01 00:09:00,40.7260,-73.9918,B02512
2,2014-08-01 00:12:00,40.7209,-74.0507,B02512
3,2014-08-01 00:12:00,40.7387,-73.9856,B02512
4,2014-08-01 00:12:00,40.7323,-74.0077,B02512
...,...,...,...,...
829270,2014-08-31 23:55:00,40.7552,-73.9753,B02764
829271,2014-08-31 23:55:00,40.7552,-73.9753,B02764
829272,2014-08-31 23:55:00,40.7617,-73.9788,B02764
829273,2014-08-31 23:59:00,40.7395,-73.9889,B02764
